In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

#Use requests to parse wikipedia page
r  = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
#Get the page as text
data = r.text

#Import data to beautiful soup
soup = BeautifulSoup(data)

#find the table
table = soup.find_all('table')[0]

#parse the data of the table
df = pd.read_html(str(table), header=0)[0]

#Clean Borough values that are not assigned
df = df[df['Borough'] != "Not assigned"]

#Translate Neighbourhood values to Borough when they are not assigned
df.loc[df['Neighbourhood'] == "Not assigned",'Neighbourhood'] = df['Borough']

#Group Neighbourhood values that belong to the same borough
df2 =  pd.DataFrame(df.groupby(['Postcode'],as_index=True)['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x)))
df.set_index('Postcode', inplace=True,)

#Drop Neighbourhood column because we will get it from the other frame
df.drop(columns=['Neighbourhood'], inplace=True)

#Join the two dataframe into one by postcode as index
df3 = df2.join(df, how='outer')

#Drop duplicates
df3.drop_duplicates(inplace=True)
df = df3[['Borough','Neighbourhood']]
print(df)
#Print results

df.shape


                   Borough                                      Neighbourhood
Postcode                                                                     
M1B            Scarborough                                     Rouge, Malvern
M1C            Scarborough             Highland Creek, Rouge Hill, Port Union
M1E            Scarborough                  Guildwood, Morningside, West Hill
M1G            Scarborough                                             Woburn
M1H            Scarborough                                          Cedarbrae
M1J            Scarborough                                Scarborough Village
M1K            Scarborough        East Birchmount Park, Ionview, Kennedy Park
M1L            Scarborough                    Clairlea, Golden Mile, Oakridge
M1M            Scarborough    Cliffcrest, Cliffside, Scarborough Village West
M1N            Scarborough                        Birch Cliff, Cliffside West
M1P            Scarborough  Dorset Park, Scarborough Town Centre

(103, 2)